<h2>MODEL TESTING</h2>

Script ini digunakan hanya untuk mengetes model-model yang telah dibuat. Tahapan dari mengetes suatu model adalah:
1. Load model yang telah ada. (Saat ini, model yang diloat adalah model v1 yang sudah saya buat).
2. Load x_test dan y_test, untuk menjadi data yang dapat dimasukkan ke model untuk melihat performa model. Bagian ini tidak perlu diubah jika model yang diload berubah.
3. Jalankan block terakhir. Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda

In [1]:
import tensorflow as tf
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import math

In [2]:
## Load the model
model = tf.keras.models.load_model('../models/model_v0.keras')

# Show the model architecture
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 60, 2)          │             5 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 60, 32)         │         4,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,605 (150.81 KB)

 Trainable params: 12,866 (50.26 KB)

 Non-trainable params: 5 (24.00 B)

 Optimizer params: 25,734 (100.53 KB)

In [3]:
## Load numpy test data
x_test = np.load('../x_test_C.npy')
y_test = np.load('../y_test_C.npy')
mean_variance = np.load('../mean_variance_C.npy')

In [4]:
## IMPORTANT: Set up denormalizer
mean = mean_variance[0]
variance = mean_variance[1]

denormalizer = tf.keras.layers.Normalization(mean=mean, variance=variance, invert=True)

In [5]:
## ONE STEP PREDICTION
# Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda
num = 43

# Predict the next coords of entry "num" using the model
num_of_timesteps = len(x_test[0])
pred_coords = denormalizer(model.predict(x_test[num].reshape(1, num_of_timesteps, 2)))
true_coords = denormalizer(y_test[num])

# Print the true and predicted coordinates
print("Prediction:")
print(pred_coords)

print("True Coordinate:")
print(true_coords)


## Map display of True Point vs Predicted Point
past_lat = []
past_lon = []

# Get route coordinates
for point in x_test[num]:
  past_lat.append(point[0])
  past_lon.append(point[1])

# Get prediction coordinates
pred_lat = pred_coords.numpy()[0][0]
pred_lon = pred_coords.numpy()[0][1]

# Get true coordinates from y
true_lat = true_coords.numpy()[0][0]
true_lon = true_coords.numpy()[0][1]


# Build the map
fig = go.Figure()

fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=past_lat,
  lon=past_lon,
  name="trip",
  marker={'size': 10}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=[pred_lat],
  lon=[pred_lon],
  name="Predicted Point",
  marker={'size': 15, 'color': 'green'}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=[true_lat],
  lon=[true_lon],
  name="True Point",
  marker={'size': 15, 'color': 'yellow'}))

fig.update_layout(mapbox_style="open-street-map",)
fig.update_layout(
  margin={"r":0,"t":0,"l":0,"b":0},
  mapbox=dict(
    zoom=10,
    center=go.layout.mapbox.Center(
            lat=-6.2,
            lon=106.816
        ),
  ))
fig.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
Prediction:
tf.Tensor([[ -6.261593 106.88707 ]], shape=(1, 2), dtype=float32)
True Coordinate:
tf.Tensor([[ -6.2617707 106.88739  ]], shape=(1, 2), dtype=float32)


In [7]:
## MULTI STEP PREDICTION
# Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda
num = 1
steps = 60

# Predict the next "step" coords of entry "num" using the model
num_of_timesteps = len(x_test[0])
x_data = x_test[num] # Data that will be fed to the model

pred_coords = []
for i in range(steps):
  current_pred = denormalizer(model.predict(x_data.reshape(1, num_of_timesteps, 2))).numpy().tolist()[0]
  pred_coords.append(current_pred)

  x_data = np.delete(x_data, 0, 0)
  
  x_data = np.append(x_data, [current_pred], axis=0)

true_coords = denormalizer(y_test[num])

# Print the true and predicted coordinates
print("Prediction:")
print(pred_coords)

print("True Coordinate:")
print(true_coords)


### Map display of True Point vs Predicted Point

# Get route coordinates
past_lat = []
past_lon = []
for point in x_test[num]:
  past_lat.append(point[0])
  past_lon.append(point[1])

# Get prediction coordinates
pred_lat = []
pred_lon = []
for point in pred_coords:
  pred_lat.append(point[0])
  pred_lon.append(point[1])

# Get true coordinates from y
true_lat = true_coords.numpy()[0][0]
true_lon = true_coords.numpy()[0][1]


# Build the map
fig = go.Figure()

# Plot previous path
fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=past_lat,
  lon=past_lon,
  name="trip",
  marker={'size': 10}))

# Plot predicted path
fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=pred_lat,
  lon=pred_lon,
  name="Predicted Point",
  marker={'size': 10, 'color': 'green'}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=[true_lat],
  lon=[true_lon],
  name="True Point",
  marker={'size': 15, 'color': 'yellow'}))

fig.update_layout(mapbox_style="open-street-map",)
fig.update_layout(
  margin={"r":0,"t":0,"l":0,"b":0},
  mapbox=dict(
    zoom=10,
    center=go.layout.mapbox.Center(
            lat=-6.2,
            lon=106.816
        ),
  ))
fig.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━